<a href="https://colab.research.google.com/github/Banafshehkh/Banafshehkh/blob/main/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import os
from scipy import sparse, linalg


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Number of Neurons
N = 279

# Cell membrane conductance (pS)
Gc = 0.1

# Cell Membrane Capacitance
C = 0.015

# Gap Junctions (Electrical, 279*279)
ggap = 1.0
Gg_Static = np.load('/content/drive/MyDrive/simulation_data/Gg.npy')

# Synaptic connections (Chemical, 279*279)
gsyn = 1.0
Gs_Static = np.load('/content/drive/MyDrive/simulation_data/Gs.npy')

# Leakage potential (mV)
Ec = -35.0

# Directionality (279*1)
E = np.load('/content/drive/MyDrive/simulation_data/emask.npy')
E = -48.0 * E
EMat = np.tile(np.reshape(E, N), (N, 1))

# Synaptic Activity Parameters
ar = 1.0 / 1.5  # Synaptic activity's rise time
ad = 5.0 / 1.5  # Synaptic activity's decay time
B = 0.125  # Width of the sigmoid (mv^-1)

# Input Mask/Continuous Transition
transit_Mat = np.zeros((2, N))
t_Tracker = 0
Iext = 100000

rate = 0.025
offset = 0.15


In [ ]:
Gg_Static

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
Gs_Static

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
E

array([[ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [-48.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [-48.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [-48.],
       [ -0.],
       [ -0.],
       [ -0.],
       [-48.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -0.],
       [ -

In [ ]:
# Connectome Arrays
Gg_Dynamic = Gg_Static.copy()
Gs_Dynamic = Gs_Static.copy()

# Data matrix stack size
stack_Size = 5
init_data_Mat = np.zeros((stack_Size + 50, N))
data_Mat = np.zeros((stack_Size, N))

In [ ]:
# Mask transition
def transit_Mask(input_Array):
    global t_Switch, oldMask, newMask, transit_End, Vth_Static, t_Tracker

    transit_Mat[0, :] = transit_Mat[1, :]

    t_Switch = t_Tracker

    transit_Mat[1, :] = input_Array

    oldMask = transit_Mat[0, :]
    newMask = transit_Mat[1, :]

    Vth_Static = EffVth_rhs(Iext, newMask)
    transit_End = t_Tracker + 0.3

    print(oldMask, newMask, t_Switch, transit_End)


In [ ]:
def update_Mask(old, new, t, tSwitch):
    return np.multiply(old, 0.5 - 0.5 * np.tanh((t - tSwitch) / rate)) + np.multiply(new, 0.5 + 0.5 * np.tanh((t - tSwitch) / rate))

In [ ]:
# Ablation
def modify_Connectome(ablation_Array):
    global Vth_Static, Gg_Dynamic, Gs_Dynamic

    apply_Col = np.tile(ablation_Array, (N, 1))
    apply_Row = np.transpose(apply_Col)

    apply_Mat = np.multiply(apply_Col, apply_Row)

    Gg_Dynamic = np.multiply(Gg_Static, apply_Mat)
    Gs_Dynamic = np.multiply(Gs_Static, apply_Mat)

    try:
        newMask
    except NameError:
        EffVth(Gg_Dynamic, Gs_Dynamic)
        if np.sum(ablation_Array) != N:
            print("Neurons " + str(np.where(ablation_Array == False)[0]) + " are ablated")
        else:
            print("All Neurons healthy")
    print("EffVth Recalculated")
    
    print("Vth Recalculated")



In [ ]:
""" Efficient V-threshold computation """
def EffVth(Gg, Gs):

  Gcmat = np.multiply(Gc, np.eye(N))
  EcVec = np.multiply(Ec, np.ones((N, 1)))
  # Add regularization term
  # regularization_term = 1e-6
  # M = M + regularization_term * np.eye(N)
  # # Print the matrix M to check its properties
  # print("Matrix M:")
  # print(M)

  M1 = -Gcmat
  b1 = np.multiply(Gc, EcVec)

  Ggap = np.multiply(ggap, Gg)
  Ggapdiag = np.subtract(Ggap, np.diag(np.diag(Ggap)))
  Ggapsum = Ggapdiag.sum(axis=1)
  Ggapsummat = np.diag(Ggapsum)
  M2 = -np.subtract(Ggapsummat, Ggapdiag)

  Gs_ij = np.multiply(gsyn, Gs)
  s_eq = round((ar / (ar + 2 * ad)), 4)
  sjmat = np.multiply(s_eq, np.ones((N, N)))
  S_eq = np.multiply(s_eq, np.ones((N, 1)))
  Gsyn = np.multiply(sjmat, Gs_ij)
  Gsyndiag = np.subtract(Gsyn, np.diag(np.diag(Gsyn)))
  Gsynsum = Gsyndiag.sum(axis=1)
  M3 = -np.diag(Gsynsum)

  b3 = np.dot(Gs_ij, np.multiply(s_eq, E))

  M = M1 + M2 + M3

  global LL, UU, bb

  (P, LL, UU) = linalg.lu(M)
  bbb = -b1 - b3
  bb = np.reshape(bbb, N)




In [ ]:
def EffVth_rhs(Iext, InMask):
  InputMask = np.multiply(Iext, InMask)
  b = np.subtract(bb, InputMask)
  Vth = np.linalg.solve(UU, np.linalg.solve(LL, b))

  return Vth


In [ ]:
def voltage_filter(v_vec, vmax, scaler):
  filtered = vmax * np.tanh(scaler * np.divide(v_vec, vmax))
  return filtered

In [ ]:
def membrane_voltageRHS(t, y):
    """ Split the incoming values """
    Vvec, SVec = np.split(y, 2)

    """ Gc(Vi - Ec) """
    VsubEc = np.multiply(Gc, (Vvec - Ec))

    """ Gg(Vi - Vj) Computation """
    Vrep = np.tile(Vvec, (N, 1))
    GapCon = np.multiply(Gg_Dynamic, np.subtract(np.transpose(Vrep), Vrep)).sum(axis=1)

    """ Gs*S*(Vi - Ej) Computation """
    VsubEj = np.subtract(np.transpose(Vrep), EMat)
    SynapCon = np.multiply(np.multiply(Gs_Dynamic, np.tile(SVec, (N, 1))), VsubEj).sum(axis=1)

    global InMask, Vth

    InMask = update_Mask(oldMask, newMask, t, t_Tracker + offset)
    Vth = EffVth_rhs(Iext, InMask)

    """ ar*(1-Si)*Sigmoid Computation """
    SynRise = np.multiply(np.multiply(ar, (np.subtract(1.0, SVec))),
                          np.reciprocal(1.0 + np.exp(-B*(np.subtract(Vvec, Vth)))))

    SynDrop = np.multiply(ad, SVec)

    """ Input Mask """
    Input = np.multiply(Iext, InMask)

    """ dV and dS and merge them back to dydt """
    dV = (-(VsubEc + GapCon + SynapCon) + Input)/C
    dS = np.subtract(SynRise, SynDrop)

    return np.concatenate((dV, dS))


In [ ]:
# import numpy as np
# from scipy.integrate import solve_ivp

# # Define the initial conditions
# V0 = np.zeros(N)  # Initial membrane voltage
# S0 = np.zeros(N)  # Initial synaptic activity
# y0 = np.concatenate((V0, S0))

# # Define the time points for the simulation
# t_start = 0  # Start time
# t_end = 10  # End time
# t_step = 0.01  # Time step
# t_points = np.arange(t_start, t_end, t_step)

# # Define global variables
# oldMask = np.zeros(N)
# newMask = np.zeros(N)
# bb = np.zeros(N)  # Define bb here
# UU = np.zeros((N, N))  # Define UU here
# LL = np.zeros((N, N))  # Define LL here

# # Call the solve_ivp function to run the simulation
# solution = solve_ivp(membrane_voltageRHS, (t_start, t_end), y0, t_eval=t_points, method='RK45')

# # Extract the results
# t = solution.t  # Time points
# V = solution.y[:N]  # Membrane voltage
# S = solution.y[N:]  # Synaptic activity

# # Print the results or perform any desired analysis
# print("Simulation results:")
# print("Time:", t)
# print("Membrane voltage:", V)
# print("Synaptic activity:", S)


In [ ]:
# Load the necessary data and configurations
EffVth(Gg_Static, Gs_Static)

# Run the network
t_Delta = 0.001
atol = 1e-6
run_Network(t_Delta, atol)

NameError: ignored